In [ ]:
!pip install sqlalchemy
!pip install psycopg2

# Paso 1: Extraccion data API

In [13]:
import pandas as pd
import psycopg2
import yfinance as yf

goo = yf.Ticker('GOOG')
# sacar la informacion historica de 1 año hacia atras
hist = goo.history(period="1y")
hist['Date']=hist.index
hist=hist.reset_index(drop=True)
hist # tenemos 8 columnas

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
0,143.399506,143.589996,140.943497,141.063004,19256000,0.0,0.0,2022-04-05 00:00:00-04:00
1,139.161499,139.848495,136.418106,137.175995,23574000,0.0,0.0,2022-04-06 00:00:00-04:00
2,136.617996,137.701508,134.857254,136.464996,19448000,0.0,0.0,2022-04-07 00:00:00-04:00
3,136.250000,136.250000,133.752502,134.010498,16434000,0.0,0.0,2022-04-08 00:00:00-04:00
4,132.899994,132.939194,129.617493,129.796494,24188000,0.0,0.0,2022-04-11 00:00:00-04:00
...,...,...,...,...,...,...,...,...
247,101.440002,101.610001,100.290001,101.320000,25009800,0.0,0.0,2023-03-30 00:00:00-04:00
248,101.709999,104.190002,101.440002,104.000000,28086500,0.0,0.0,2023-03-31 00:00:00-04:00
249,102.669998,104.949997,102.379997,104.910004,20719900,0.0,0.0,2023-04-03 00:00:00-04:00
250,104.839996,106.099998,104.599998,105.120003,20367700,0.0,0.0,2023-04-04 00:00:00-04:00


# Paso 2: Limpieza de data

In [14]:
hist=hist.rename(columns={'Open':'O','High':'H','Low':'L','Close':'C','Volume':'V',\
                          'Dividends':'D','Stock Splits':'S', 'Date':'Dat'})
hist= hist.drop(columns=['Dat'])
hist

,O,H,L,C,V,D,S
0,143.399506,143.589996,140.943497,141.063004,19256000,0.0,0.0
1,139.161499,139.848495,136.418106,137.175995,23574000,0.0,0.0
2,136.617996,137.701508,134.857254,136.464996,19448000,0.0,0.0
3,136.250000,136.250000,133.752502,134.010498,16434000,0.0,0.0
4,132.899994,132.939194,129.617493,129.796494,24188000,0.0,0.0
...,...,...,...,...,...,...,...
247,101.440002,101.610001,100.290001,101.320000,25009800,0.0,0.0
248,101.709999,104.190002,101.440002,104.000000,28086500,0.0,0.0
249,102.669998,104.949997,102.379997,104.910004,20719900,0.0,0.0
250,104.839996,106.099998,104.599998,105.120003,20367700,0.0,0.0


# Paso 3: Crear la tabla de destino en redshift

```SQL
CREATE TABLE stock_prices (
  O decimal(10,2),
  H decimal(10,2),
  L decimal(10,2),
  C decimal(10,2),
  V bigint,
  D decimal(10,2),
  S decimal(10,2)
);
```

# Paso 4: establecer conexion a Redshift

In [15]:
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws"
data_base="data-engineer-database"
user="dafbustosus_coderhouse"
with open("C:/Users/Windows/Downloads/pwd_coder.txt",'r') as f:
    pwd= f.read()

In [16]:
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [17]:
conn

<connection object at 0x000001D561A209D0; dsn: 'user=dafbustosus_coderhouse password=xxx dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port=5439', closed: 0>

# Paso 5: Verificar que la tabla exista

In [18]:
cur = conn.cursor()
# Execute a SQL query to select data from a table
cur.execute("SELECT * FROM stock_prices")
# Fetch the results
results = cur.fetchall()
results

[]

# Paso 6: Enviar resultados de tabla a redshift

In [19]:
hist.head()

,O,H,L,C,V,D,S
0,143.399506,143.589996,140.943497,141.063004,19256000,0.0,0.0
1,139.161499,139.848495,136.418106,137.175995,23574000,0.0,0.0
2,136.617996,137.701508,134.857254,136.464996,19448000,0.0,0.0
3,136.250000,136.250000,133.752502,134.010498,16434000,0.0,0.0
4,132.899994,132.939194,129.617493,129.796494,24188000,0.0,0.0


In [20]:
from psycopg2.extras import execute_values
cur = conn.cursor()
# Define the table name
table_name = 'stock_prices'
# Define the columns you want to insert data into
columns = ['O', 'H', 'L','C','V','D','S']
# Generate 
values = [tuple(x) for x in hist.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
# Execute the INSERT statement using execute_values
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")
# Close the cursor and connection
#cur.close()
#conn.close()

In [21]:
cur.close()
conn.close()